<a href="https://colab.research.google.com/github/GorokhovSemyon/TextClassification/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
!pip install peft langchain

In [4]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from langchain.prompts import PromptTemplate

adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"

tokenizer = AutoTokenizer.from_pretrained(
              base_model_name,
              trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
device_map = {"": 0}

model = AutoPeftModelForCausalLM.from_pretrained(
              adapt_model_name,
              device_map=device_map,
              torch_dtype=torch.bfloat16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

In [5]:
# Load model from HuggingFace Hub
sent_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
sent_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [6]:
def get_embedding(sentence):

    #Mean Pooling - Take attention mask into account for correct averaging
    def _mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Tokenize sentences
    encoded_input = sent_tokenizer([sentence], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = sent_model(**encoded_input)

    # Perform pooling
    sentence_embeddings = _mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings

In [7]:
answers = []
emb_database = torch.empty((0, 384), dtype=torch.float32)

In [53]:
info_prompt_less10 = PromptTemplate.from_template("user: Модернизированная лаборатория по анализу данных, расположенная на улице Иванова 10, в городе Новоград. Количество патентов - 21. Лаборатория специализируется на обработке и анализе данных различных форматов и объемов, занимается исследованиями в области LLM и NLP. Вакансия: Аналитик данных - зарплата 200 тысяч рублей в месяц. Требования: Владение Python и SQL, опыт работы с большими данными. Кандидат должен пройти тестирование, интервью с командой проекта. Для сотрудников лаборатории доступны бонусы: Корпоративное обучение, Гибкий график работы. Лаборатория проводит командные мероприятия и конференции. {question}\nbot: Вот краткий ответ на ваш вопрос, содержащий не более 10 слов:")

In [54]:
def get_answer(info_prompt, question):

    prompt = info_prompt.format(question=question)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                            top_p=0.5,
                            temperature=0.3,
                            attention_mask=inputs["attention_mask"],
                            max_new_tokens=50,
                            pad_token_id=tokenizer.eos_token_id,
                            do_sample=True)

    output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    parsed_answer = output.split("Вот краткий ответ на ваш вопрос, содержащий не более 10 слов:")[1].strip()

    if "bot:" in parsed_answer:
        parsed_answer = parsed_answer.split("bot:")[0].strip()

    return parsed_answer

In [55]:
question = "Каких работников вы ищете?"
emb = get_embedding(question)
emb

tensor([[ 2.5110e-02,  5.1584e-03, -6.1139e-02, -1.9740e-02, -4.8976e-02,
          5.2549e-02,  1.1428e-01,  1.3328e-01, -2.0873e-02, -6.2296e-02,
         -2.0364e-03,  3.7444e-02,  1.4026e-02,  8.9859e-02, -1.1850e-02,
          6.5547e-03, -6.0301e-03, -1.9408e-03,  1.1191e-02,  2.6632e-02,
         -2.8001e-02, -3.1463e-02,  5.4731e-02,  1.0286e-01, -5.3318e-03,
         -4.4906e-03,  2.8889e-02, -1.7661e-02,  4.1668e-02,  1.5056e-02,
         -2.3516e-04, -2.2797e-02,  1.1825e-02, -7.4655e-03,  3.4711e-02,
          6.5544e-05, -1.1831e-02,  1.7795e-02,  5.8787e-02,  2.8625e-02,
         -4.6337e-02, -7.2969e-02, -7.6595e-02,  8.9365e-02,  1.0229e-02,
          1.1044e-01, -4.5784e-02,  6.8168e-03,  3.7613e-02, -1.0416e-02,
         -1.0604e-01, -9.1198e-03, -3.0156e-02,  6.5540e-02,  5.0515e-02,
         -9.9385e-02, -1.8042e-02,  1.1136e-02, -1.0821e-01, -6.7553e-02,
          6.4620e-02, -3.5007e-02, -3.5723e-02, -3.0226e-02,  1.3148e-02,
          9.5499e-03,  5.0622e-02, -5.

In [56]:
def get_cos_sim(question):
    cos_sim = F.cosine_similarity(emb_database, emb, dim=1, eps=1e-8)
    return cos_sim

get_cos_sim(question)

tensor([1.0000, 1.0000, 0.6325, 0.6885, 0.6541, 0.6140, 0.8262, 0.6506, 0.6224,
        0.6768, 0.5531, 1.0000, 0.6325, 0.7424])

In [57]:
answer = get_answer(info_prompt_less10, question)
emb_database = torch.cat((emb_database, emb), 0)
answers.append(answer)
print(f'Answer from model: {answer}')

Answer from model: Мы ищем аналитиков данных.


In [58]:
question = "Где находится ваша лаборатория?"
emb = get_embedding(question)
cos_sim = get_cos_sim(question)
cos_sim

tensor([0.6325, 0.6325, 1.0000, 0.6934, 0.8163, 0.7052, 0.5991, 0.5740, 0.5769,
        0.6009, 0.5323, 0.6325, 1.0000, 0.6858, 0.6325])

In [59]:
answer = get_answer(info_prompt_less10, question)
emb_database = torch.cat((emb_database, emb), 0)
answers.append(answer)
print(f'Answer from model: {answer}')

Answer from model: Лаборатория находится на улице Иванова 10 в городе Новоград.


In [60]:
question = "Где расположен ваша лаборатория?"
emb = get_embedding(question)
cos_sim = get_cos_sim(question)
cos_sim

tensor([0.6342, 0.6342, 0.8826, 0.6612, 0.8823, 0.6914, 0.6007, 0.5501, 0.5847,
        0.6653, 0.5767, 0.6342, 0.8826, 0.7292, 0.6342, 0.8826])

In [61]:
print(f'{cos_sim} {answers=}')

tensor([0.6342, 0.6342, 0.8826, 0.6612, 0.8823, 0.6914, 0.6007, 0.5501, 0.5847,
        0.6653, 0.5767, 0.6342, 0.8826, 0.7292, 0.6342, 0.8826]) answers=['Мы ищем аналитиков данных.', 'Мы ищем аналитиков данных.', 'Лаборатория находится на улице Иванова 10 в городе Новоград.', 'Улица Иванова 10, город Новоград.', 'Лаборатория расположена на улице Иванова 10, в городе Новоград.', 'Анализ данных, обработка данных, большие данные, Python, SQL, корпоративное обучение, гибкий график работы, командные мероприя', 'Мы ищем специалистов в области анализа данных.', '- Python\n- SQL\n- большие данные\n- корпоративное обучение\n- гибкий график работы\n- командные мероприятия\n- конферен', '200 тысяч рублей в месяц.', 'Присылайте резюме и пройдите интервью.', 'Корпоративное обучение, гибкий график работы, командные мероприятия, конференции.', 'Мы ищем аналитиков данных с опытом работы с большими данными.', 'Лаборатория находится на улице Иванова 10 в городе Новоград.', '21 патентов.', 'Мы ищем анал

In [65]:
questions = [
"Какой адрес вашей лаборатории данных?",
"Где расположена ваша лаборатория данных?",
"Каково местоположение вашей лаборатории данных?",
"Где конкретно находится ваша лаборатория данных?",
"Чем занимается ваша лаборатория данных?",
"Какие работы проводятся в вашей лаборатории данных?",
"Какие анализы и исследования выполняются в вашей лаборатории данных?",
"На что направлена деятельность вашей лаборатории данных?",
"Какие анализы и данные обрабатываются в вашей лаборатории данных?",
"Какие услуги и продукты предоставляет ваша лаборатория данных?",
"Каких специалистов вы ищете для работы в лаборатории данных?",
"Какие специалисты вам нужны в настоящее время для работы в лаборатории данных?",
"Какие квалификации и навыки требуются у специалистов для работы в вашей лаборатории данных?",
"Какими навыками и знаниями должны обладать специалисты для успешной работы в лаборатории данных?",
"Какие специфические компетенции и опыт необходимы для специалистов в вашей лаборатории данных?",
"Какая заработная плата предложена для аналитика данных?",
"Какая оплата труда предусмотрена для аналитика данных?",
"Какой оклад установлен для должности аналитика данных?",
"Как устроиться на должность аналитика данных в вашей лаборатории?",
"Как можно получить работу аналитиком данных в вашей лаборатории?",
"Каков процесс найма на позицию аналитика данных в вашей лаборатории?",
"Сколько у вашей лаборатории патентов?"]

In [66]:
for q in questions:
    print(q)
    emb = get_embedding(q)
    cos_sim = get_cos_sim(q)
    max_value, max_index = torch.max(get_cos_sim(q), dim=0)

    if max_value > 0.9:
        answer = answers[max_index]
        print(f'DATABASE: {answer}')
    else:
        answer = get_answer(info_prompt_less10, q)
        emb_database = torch.cat((emb_database, emb), 0)
        answers.append(answer)
        print(f'MODEL: {answer}')
    print()

Какой адрес вашей лаборатории данных?
DATABASE: Улица Иванова 10, город Новоград.

Где расположена ваша лаборатория данных?
DATABASE: Лаборатория расположена на улице Иванова 10, в городе Новоград.

Каково местоположение вашей лаборатории данных?
DATABASE: Улица Иванова 10, город Новоград.

Где конкретно находится ваша лаборатория данных?
DATABASE: Лаборатория расположена на улице Иванова 10, в городе Новоград.

Чем занимается ваша лаборатория данных?
DATABASE: Анализ данных, LLM, NLP, большие данные, Python, SQL.

Какие работы проводятся в вашей лаборатории данных?
DATABASE: Анализ данных, LLM, NLP, обработка данных, исследования, большие данные.

Какие анализы и исследования выполняются в вашей лаборатории данных?
DATABASE: Анализ данных, LLM, NLP, большие данные, исследования, обработка данных.

На что направлена деятельность вашей лаборатории данных?
DATABASE: Анализ данных, LLM, NLP, исследования, большие данные.

Какие анализы и данные обрабатываются в вашей лаборатории данных?
D